In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.api as sm
import math

In [2]:
# import all the maximals from the other file
dfMaximal1 = pd.read_csv("./playlistData/maximal1_T4_50.csv")
#dfMaximal2 = pd.read_csv("./playlistData/maximal2_T4_50.csv")
#dfMaximal3 = pd.read_csv("./playlistData/maximal3_T4_50.csv")
#dfMaximal4 = pd.read_csv("./playlistData/maximal4_T4_50.csv")
#dfMaximal5 = pd.read_csv("./playlistData/maximal5_T5_50.csv")
#maximals = [dfMaximal1, dfMaximal2, dfMaximal3, dfMaximal4, dfMaximal5]
maximals = [dfMaximal1]

# just using to get sample playlists instead of manually typing out songs
dfPlaylists = pd.read_csv("./playlistData/spotify_dataset.csv", on_bad_lines='skip')

In [3]:
# passing tuples turned them into strings
# "("item1", "item2")" -> ("item1", "item2")
def parseTuple(songAndArtist):
    artist, song = "", ""
    foundOpen, foundComma, foundSpace, foundClose = False, False, False, False
    for i in songAndArtist:
        if (not foundOpen and i == "("):
            foundOpen = True
            continue
            
        if (not foundComma and i == ","):
            foundComma = True
            continue
            
        if (not foundSpace and i == " "):
            foundSpace = True
            continue
            
        if (not foundClose and i == ")"):
            foundClose = True
        
        if (foundOpen and not foundComma and i != "'"):
            artist += i
            continue
            
        if (foundComma and foundSpace and not foundClose and i != "'"):
            song += i
            continue
            
        
            
        if (foundClose and artist[0] == " "):
            artist = artist[1:-1]
            
        if (foundClose and song[0] == " "):
            song = song[1:-1]
        
    return (artist, song)

In [4]:
# turns the maximal dataframe into libraries for quick lookup
def parseMaximal(dfMaximals):
    quadIntrest = {}
    tripleIntrest = {}
    pairIntrest = {}
    maximalsLength = dfMaximals.shape[0]
    count = 0
    for i in range(maximalsLength):
        if (not dfMaximals.isnull().iloc[i][3]):
            if (not dfMaximals.isnull().iloc[i][4]):
                # quads
                quad = [parseTuple(dfMaximals.iloc[i][1]), parseTuple(dfMaximals.iloc[i][2]), parseTuple(dfMaximals.iloc[i][3]), parseTuple(dfMaximals.iloc[i][4])]
                for j in range(4):
                    # (1, 2, 3) -> 4
                    subsection = quad.copy()
                    subsection.remove(quad[j])
                    key = (tuple(subsection), quad[j])
                    quadIntrest[key] = dfMaximals.iloc[i][j + 5]

                    # (1, 2) -> 3
                for k in range(4):
                    if (k == 0):
                        sub1 = [quad[0], quad[1], quad[2]]
                    elif (k == 1):
                        sub1 = [quad[0], quad[1], quad[3]]
                    elif (k == 2):
                        sub1 = [quad[0], quad[2], quad[3]]
                    elif (k == 3):
                        sub1 = [quad[1], quad[2], quad[3]]

                    for j in sub1:
                        subsection = sub1.copy()
                        subsection.remove(j)
                        key = (tuple(subsection), j)
                        # if key already exists, the higher intrest score is used
                        if (key in list(tripleIntrest.keys())):
                            currentIntrest = tripleIntrest[key]
                            if (currentIntrest < dfMaximals.iloc[i][k + 9]):
                                tripleIntrest[key] = dfMaximals.iloc[i][k + 9]
                        else:
                            tripleIntrest[key] = dfMaximals.iloc[i][k + 9]

                # (1) -> 2
                for k in range(6):
                    if (k == 0):
                        sub2 = [quad[0], quad[1]]
                    elif (k == 1):
                        sub2 = [quad[0], quad[2]]
                    elif (k == 2):
                        sub2 = [quad[0], quad[3]]
                    elif (k == 3):
                        sub2 = [quad[1], quad[2]]
                    elif (k == 4):
                        sub2 = [quad[1], quad[3]]
                    elif (k == 5):
                        sub2 = [quad[2], quad[3]]

                    for j in sub2:
                        subsection = sub2.copy()
                        subsection.remove(j)
                        key = (subsection[0], j)
                        if (key in list(pairIntrest.keys())):
                            currentIntrest = pairIntrest[key]
                            if (currentIntrest < dfMaximals.iloc[i][k + 25]):
                                pairIntrest[key] = dfMaximals.iloc[i][k + 25]
                        else:
                            pairIntrest[key] = dfMaximals.iloc[i][k + 25]

            #triples
            triple = [parseTuple(dfMaximals.iloc[i][1]), parseTuple(dfMaximals.iloc[i][2]), parseTuple(dfMaximals.iloc[i][3])]
            for j in range(3):
                # (1, 2) -> 3
                subsection = triple.copy()
                subsection.remove(triple[j])
                key = (tuple(subsection), triple[j])
                if (key in list(tripleIntrest.keys())):
                    currentIntrest = tripleIntrest[key]
                    if (currentIntrest < dfMaximals.iloc[i][j + 5]):
                        tripleIntrest[key] = dfMaximals.iloc[i][j + 5]
                else:
                    tripleIntrest[key] = dfMaximals.iloc[i][j + 5]

            # (1) -> 2
            for k in range(3):
                if (k == 0):
                    sub2 = [triple[0], triple[1]]
                elif (k == 1):
                    sub2 = [triple[0], triple[2]]
                elif (k == 2):
                    sub2 = [triple[1], triple[2]]

                for j in sub2:
                    subsection = sub2.copy()
                    subsection.remove(j)
                    key = (subsection[0], j)
                    if (key in list(pairIntrest.keys())):
                        currentIntrest = pairIntrest[key]
                        if (currentIntrest < dfMaximals.iloc[i][k + 8]):
                            pairIntrest[key] = dfMaximals.iloc[i][k + 8]
                    else:
                        pairIntrest[key] = dfMaximals.iloc[i][k + 8]

        #pairs
        pair = [parseTuple(dfMaximals.iloc[i][1]), parseTuple(dfMaximals.iloc[i][2])]
        for j in range(2):
            # (1) -> 2
            subsection = pair.copy()
            subsection.remove(pair[j])
            key = (subsection[0], pair[j])
            if (key in list(pairIntrest.keys())):
                currentIntrest = pairIntrest[key]
                if (currentIntrest < dfMaximals.iloc[i][j + 5]):
                    pairIntrest[key] = dfMaximals.iloc[i][j + 5]
            else:
                pairIntrest[key] = dfMaximals.iloc[i][j + 5]


    return quadIntrest, tripleIntrest, pairIntrest

In [5]:
# create list of association rule, intrest value dictionaries
quadIntrestList, tripleIntrestList, pairIntrestList = [], [], []
for i in maximals:
    quadIntrestHold, tripleIntrestHold, pairIntrestHold = parseMaximal(i)
    print("Returned")
    quadIntrestList.append(quadIntrestHold)
    tripleIntrestList.append(tripleIntrestHold)
    pairIntrestList.append(pairIntrestHold)

Returned


In [6]:
pairIntrest = pairIntrestList[0]
tripleIntrest = tripleIntrestList[0]
quadIntrest = quadIntrestList[0]
quadKeys = list(quadIntrest.keys())
tripleKeys = list(tripleIntrest.keys())
pairKeys = list(pairIntrest.keys())
print(len(quadKeys))
print(len(tripleKeys))
print(len(pairKeys))
print(pairKeys[0][0])

23296
7629
2490
('CrystalCastles', 'Celestic')


In [7]:
# use a maximal to generate recommendations
def GenerateRecomendations(playlist, maximalNum):
    pairIntrest = pairIntrestList[maximalNum]
    tripleIntrest = tripleIntrestList[maximalNum]
    quadIntrest = quadIntrestList[maximalNum]
    count = 0
    recommendations = []
    
    # (1, 2, 3) -> 4
    # sets are (1, 2, 3), rules are 4
    pairSets = []
    pairRules = []
    tripleSets = []
    tripleRules = []
    quadSets = []
    quadRules = []
    for i in pairKeys:
        pairSets.append(i[0])
        pairRules.append(i[1])
    for i in tripleKeys:
        tripleSets.append(i[0])
        tripleRules.append(i[1])
    for i in quadKeys:
        quadSets.append(i[0])
        quadRules.append(i[1])
    
    # find all single for pair rules
    for i in range(len(pairSets)):
        single = pairSets[i]
        rule = pairRules[i]
        
        if (single in playlist):
            recExists = False
            # check if it has already been recommended, if so see if you can increase the intrest
            for j in recommendations:
                if (rule == j[0]):
                    recExists = True
                    if (pairIntrest[(single, rule)] > j[1]):
                        recommendations.append((rule, pairIntrest[(single, rule)]))
            if (not recExists):
                recommendations.append((rule, pairIntrest[(single, rule)]))
                
    # do singles first so you can make this check,
    # if it succeeds, the triples or quads are not 
    # going to find anything either
    if (len(recommendations) == 0):
        return []
                
    # find all pairs for triple rules
    for i in range(len(tripleSets)):
        pair = tripleSets[i]
        rule = tripleRules[i]
        
        if (pair[0] in playlist and pair[1] in playlist):
            recExists = False
            for k in recommendations:
                if (rule == k[0]):
                    recExists = True
                    if (tripleIntrest[(pair, rule)] > k[1]):
                        recommendations.append((rule, tripleIntrest[(pair, rule)]))
            if (not recExists):
                recommendations.append((rule, tripleIntrest[(pair, rule)]))
                
                
    # find all triples for quad rules
    for i in range(len(quadSets)):
        triple = quadSets[i]
        rule = quadRules[i]
        
        if (triple[0] in playlist and triple[1] in playlist and triple[2] in playlist):
            recExists = False
            for k in recommendations:
                if (rule == k[0]):
                    recExists = True
                    if (quadIntrest[(triple, rule)] > k[1]):
                        recommendations.append((rule, quadIntrest[(pair, rule)]))
            if (not recExists):
                recommendations.append((rule, quadIntrest[(pair, rule)]))
                    
    return recommendations

In [8]:
# load a couple sample playlists to test

playlists = []
playlistNames = []
dfPlaylists.iloc[0][3]
for i in range(600): # change to the whole thing at end
    currentPLName = dfPlaylists.iloc[i][3]
    if (currentPLName not in playlistNames):
        playlistNames.append(currentPLName)
        playlists.append([])
        
    playlists[playlistNames.index(currentPLName)].append((dfPlaylists.iloc[i][1], dfPlaylists.iloc[i][2]))
    
print(len(playlists))

10


In [9]:
rec = GenerateRecomendations(playlists[7], 0)

In [10]:
pendulumCount = 0
crystalCastlesCount = 0
justiceCount = 0
theProdigyCount = 0
for i in playlists[7]:
    if (i[0] == 'Pendulum'):
        pendulumCount += 1
    if (i[0] == 'Crystal Castles'):
        crystalCastlesCount += 1
    if (i[0] == 'Justice'):
        justiceCount += 1
    if (i[0] == 'The Prodigy'):
        theProdigyCount += 1

print("Pendulum Count: ", pendulumCount, " Crystal Castles Count: ", crystalCastlesCount, "Justice Count: ", justiceCount, "The Prodigy Count: ", theProdigyCount)
print(playlists[7])
print()
print(rec)

Pendulum Count:  10  Crystal Castles Count:  9 Justice Count:  15 The Prodigy Count:  11
[('Crystal Castles', '1991'), ('SomethingALaMode', '5 AM'), ('Pendulum', '9,000 Miles'), ('Data', 'Aerius Light (Kitsuné Dj-Friendly Edit)'), ('Crystal Castles', 'Air War - David Wolf Remix'), ('Crystal Castles', 'Alice Practice'), ('Ratatat', 'Alps'), ('Crystal Castles', 'Baptism'), ('Ratatat', 'Bare Feast'), ('Ratatat', 'Bilar'), ('Ratatat', 'Bird-Priest'), ('Ratatat', 'Black Heroes'), ('Ratatat', 'Bob Gandhi'), ('Ratatat', 'Breaking Away'), ('Ratatat', 'Bruleé'), ('Ratatat', 'Bustelo'), ('Crystal Castles', 'Celestica'), ('Ratatat', 'Cherry'), ('The Prodigy', 'Colours'), ('Crystal Castles VS Health', 'Crimewave'), ('Ratatat', 'Crips'), ('Justice', 'D.A.N.C.E'), ('Justice', 'D.A.N.C.E - MSTRKRFT Remix'), ('Justice', 'D.A.N.C.E. - Radio Edit'), ('Justice', 'DVNO'), ('Ratatat', 'Desert Eagle'), ('Pendulum', 'Different'), ('Glass Candy', 'Digital Versicolor'), ('Ratatat', 'Drugs'), ('Ratatat', 'Dura'